In [1]:
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm as tqdm
from keras.models import load_model
from keras import backend as K
from datetime import datetime

from Dental_Tool.Data_processing import *
from Dental_Tool.Process_results import *
from Dental_Tool.Dental_Model import *
from Dental_Tool.Dataloader import *
from Dental_Tool.KFold_v2 import *

import keras.backend as K 
import matplotlib.pyplot as plt
import numpy as np
import keras
import time

Using TensorFlow backend.


In [2]:
root_dir = "Results/Emsemble_5_Fold______interdental20200909-170725_Max"

In [3]:
all_models = { "CNN_Net": CNN_Net, "Inception": Inception, "ResNet": ResNet }
performance_list = []
fold_num = 5

for i in range(1, fold_num):
        fold = f"Fold_{i}"
        K.clear_session()
        
        best_models = []
        num_params = 1
        test_dataset = pd.read_csv(f"{root_dir}/{fold}/table/test_dataset.csv")
        test_generator = make_generator(test_dataset)

        for model_name, model_fnc in all_models.items():
                idx = 0
                K.clear_session()
                param_dir = model_name + f"_Fold_{i}" # % fold_data_num
                top_k_param = get_k_top_value(f"{root_dir}/{fold}/{param_dir}/parameter",k_th = num_params )
                best_models += top_k_param
#                 best_acc = -99.0
#                 best_param = ""

#                 for param in top_k_param:
#                         K.clear_session()
#                         print("loading %s ......" % param)
#                         start = time.time()
#                         model = load_model(param)
#                         end = time.time()
#                         elapse = end - start
#                         print("loading Done, cost %f seconds" %  elapse )

#                         pred = model.predict_generator(test_generator, verbose=1)
#                         pred = np.argmax(pred, axis=1)

#                         # change 
#                         pred = pred[:len(test_dataset)]
#                         y = test_dataset.Class.astype('int')

#                         confusion = confusion_matrix(y, pred)
#                         curr_acc = (pred == y).mean() * 100
#                         if curr_acc > best_acc: 
#                                 best_param = param
#                                 best_acc = curr_acc
#                         title = "Accuracy  = {:5.2f} %".format(curr_acc)
#                         print(title)

#                         idx += 1
#                         K.clear_session()

#                 best_models.append( (best_param, best_acc) )

        test_results, test_prob = [], []
        K.clear_session()
        for best_param_path in best_models:
                K.clear_session()
                print("loading %s ......" % best_param_path)
                start = time.time()
                best_model = load_model(best_param_path)
                end = time.time()
                elapse = end - start
                print("loading Done, cost %f seconds" % (elapse) )

                test_predict = best_model.predict_generator(test_generator, verbose=1)
                test_predict = test_predict[:len(test_dataset)]
                
                test_prob.append(test_predict)
                
                test_result  = np.argmax(test_predict, axis=1)
                y = test_dataset.Class.astype('int')
                accuracy = (test_result == y).mean() * 100
                title = "Accuracy  = {:5.2f} %".format(accuracy)
                print(title)
                test_results.append(test_result)
                K.clear_session()


        test_dataset_vote  = vote(test_dataset , test_results, list(all_models.keys()) )
        confusion = confusion_matrix(test_dataset_vote.Class.astype('int'), test_dataset_vote.Predict.astype('int'))
        curr_acc = (test_dataset_vote.Class == test_dataset_vote.Predict).mean() * 100
        title = "Accuracy  = {:5.2f} %".format(curr_acc)
        print(title)

        #         plot_confusion_matrix(confusion, len(np.unique(test_dataset.Class)), curr_acc, "Results/%s/Fold_%d" % (root_dir, fold_data_num), 0)
        
        # sum of probability
        test_prob = np.around(test_prob, decimals=2)
        test_dataset_vote["CNN_Prob"]       = pd.Series(test_prob[0].tolist())
        test_dataset_vote["Inception_Prob"] = pd.Series(test_prob[1].tolist())
        test_dataset_vote["Resnet_Prob"]    = pd.Series(test_prob[2].tolist())
        test_dataset_vote["Sum of Probability"] = pd.Series(np.sum(test_prob, axis=0).tolist())

        vote_by_prob = np.argmax(np.sum(test_prob, axis=0), axis=1)
        test_dataset_vote["Predict_prob"] = vote_by_prob
        confusion_prob = confusion_matrix(test_dataset_vote.Class.astype('int'), vote_by_prob)
        curr_acc_prob = (test_dataset_vote.Class == vote_by_prob ).mean() * 100
        title_prob = "Accuracy  = {:5.2f} %".format(curr_acc_prob)
        print("Vote %s, Probability %s" %(title, title_prob))

        #         plot_confusion_matrix(confusion_prob, len(np.unique(test_dataset.Class)), curr_acc_prob, "Results/%s/Fold_%d" % (root_dir, fold_data_num), 0)
        test_dataset_vote.to_csv(f"{root_dir}/{fold}/table/test_predict_vote.csv", index=False)
        print(statistic(test_dataset_vote))
        K.clear_session()

        performance_list.append(curr_acc)

loading Results/Emsemble_5_Fold______interdental20200909-170725/Fold_1/CNN_Net_Fold_1/parameter\weights-improvement-15-0.7122.hdf5 ......
loading Done, cost 6.455030 seconds
1088/1088 [==============================] - 67s 61ms/step
Accuracy  = 65.03 %
loading Results/Emsemble_5_Fold______interdental20200909-170725/Fold_1/Inception_Fold_1/parameter\weights-improvement-14-0.7435.hdf5 ......
loading Done, cost 453.933509 seconds
1088/1088 [==============================] - 45s 41ms/step
Accuracy  = 68.32 %
loading Results/Emsemble_5_Fold______interdental20200909-170725/Fold_1/ResNet_Fold_1/parameter\weights-improvement-11-0.7422.hdf5 ......
loading Done, cost 160.286903 seconds
1088/1088 [==============================] - 54s 50ms/step
Accuracy  = 68.88 %
# of no majority: 759
Accuracy  = 70.23 %
Vote Accuracy  = 70.23 %, Probability Accuracy  = 70.11 %
  Class 0 Class 1 Class 2  Acc
0    5528     239      33  95%
1    3940    1731     129  68%
2    2640    7849    1111  68%
3     417   

KeyboardInterrupt: 